In [ ]:
import sys
import os
import tifffile
import pprint
import numpy as np
from pathlib import Path
from itkwidgets import view, compare
from matplotlib.pyplot import imshow, subplots
from careamics_portfolio import PortfolioManager

from careamics_restoration.engine import Engine
from careamics_restoration.metrics import psnr

Import Dataset Portfolio


In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

Read the specific dataset 
bla 

In [ ]:
# Download files
root_path = Path("data")
files = portfolio.denoising.Flywing.download(root_path)
print(f"List of downloaded files: {files}")

Initialize the Engine

In [ ]:
engine = Engine("n2v_3D.yml")

Visualize tiles ? Model summary ?

In [ ]:
import torchsummary

torchsummary.summary(engine.model.cuda(), (1, 32, 64, 64))



Start training

In [ ]:
engine.train()

Predict

In [ ]:
preds = engine.predict()

Visualize


In [ ]:
# Get the path from config ?
original_image = tifffile.imread('data/flywing-data/flywing.tif')

In [ ]:
view(original_image)

In [ ]:
view(preds.squeeze())

In [ ]:
compare(original_image, preds.squeeze())